# Decision making in high uncertainty environments : a behavioural analog to BCI Training

This notebook runs simulation for our behavioural experiment.

In [1]:
import numpy as np


import actynf
print(actynf.__version__)


# ENVIRONMENTAL CONSTANTS :
GRID_SIZE = (7,7)
START_COORD = [[5,1],[5,2],[4,1]]
END_COORD = [0,6]

T = 10
Th = 2
action_cap = 3
state_cap = 3
structure_hypothesis = actynf.LINEAR    # Assuming that the state space has a "linear" structure allows the agent to generalize its 
                                        # finding from one state to the next, making learning easier. This only affects action learning
generalize_temperature = 2.5


0.1.39


In [19]:
from actynf import layer,link,layer_network
from models import behavioural_process,naive_model,basic_latent_model,grid_latent_model

# DEFINING THE ENVIRONMENT : 
def get_task_environment(N_fb_ticks):
    A,B,C,D,E,U = behavioural_process(GRID_SIZE,START_COORD,END_COORD,N_fb_ticks)
    process_layer = layer("environment","process",A,B,C,D,E,U,T,Th)
    return process_layer

def taks_model(model_parameters,modeltype="naive"):    
    if modeltype=="naive":
        model_builder = naive_model
    elif modeltype=="1d":
        model_builder = basic_latent_model
    elif modeltype=="2d":
        model_builder = grid_latent_model
    else:
        raise NotImplementedError("Model type not implemented")
    
    process_layer = get_task_environment(model_parameters["N_feedback_ticks"])
    layers = [process_layer]
    
    for action_model in ["angle","position","distance"]:
        a,b,c,d,e,u = model_builder(model_parameters,action_model)
        this_layer = layer("subject_"+modeltype+"_"+action_model,"model",a,b,c,d,e,u,T,Th)
        
        # layer additionnal options : 
        this_layer.hyperparams.alpha = 32
        this_layer.hyperparams.cap_action_explo = action_cap
        this_layer.hyperparams.cap_state_explo = state_cap

        this_layer.learn_options.eta = 10
        this_layer.learn_options.learn_a = False
        this_layer.learn_options.learn_b = True
        this_layer.learn_options.learn_d = True
        
        # Agents will assume that their actions have similar effects whatever their starting states
        this_layer.learn_options.assume_state_space_structure = structure_hypothesis
        this_layer.learn_options.generalize_fadeout_function_temperature = generalize_temperature
        
        this_layer.inputs.o = link(process_layer, (lambda l: l.o))
        layers.append(this_layer)
    
    # Only distance and angle actually affect the environment !
    angle_model = layers[0]
    distance_model = layers[2]
    process_layer.inputs.u = link([angle_model,distance_model], (lambda l_ang,l_dist: (l_ang.u if (l_dist.u != 0) else np.array([0]))))
    
    expe_network = layer_network(layers,"network_"+modeltype)
    return expe_network




In [20]:
parameters = {
    "N_feedback_ticks":5,
    "b_str_init":0.1,
    "d_str_init":0.1,
    "Ns_latent":5,
    "grid_size":[5,5]
}

network_grid = taks_model(parameters,"naive")

print(network_grid.layers[0])
print(network_grid.layers[1])
Ntrials = 10
STMs,weights = network_grid.run_N_trials(Ntrials)

LAYER environment : 
 -------------------------------------
LAYER DIMENSION REPORT (environment): 

Observation modalities : 1
    Modality 0 : 5 outcomes.
Hidden states factors : 1
    Model factor 0 : 49 possible states. 
Number of potential actions : 9
    Factor 0 : 9 possible transitions. 
-------------------------------------

##################################################
Layer weights :
   Matrix a :
     Modality 0 :
[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.16 0.   0.   0.   0.   0.   0.
  0.4  0.02 0.   0.   0.   0.   0.   0.68 0.33 0.02 0.   0.   0.   0.
  1.   0.68 0.4  0.16 0.   0.   0.  ]
 [0.83 0.36 0.   0.   0.   0.   0.   0.87 0.4  0.   0.   0.   0.   0.
  0.98 0.54 0.11 0.   0.   0.   0.   0.84 0.75 0.36 0.   0.   0.   0.
  0.6  0.98 0.67 0.36 0.11 0.   0.   0.32 0.67 0.98 0.75 0.54 0.4  0.36
  0.   0.32 0.6  0.84 0.98 0.87 0.83]
 [0.17 0.64 0.89 0.41 0.   0.   0.   0.13 0.6  0.94 0.49 0.05 0.   0

RuntimeError: ERROR in network <network_naive> - layer [environment] :